In [ ]:
%pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access your API key
api_key = os.getenv("GEMINI_API_KEY")

In [4]:
from google import genai

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Calculate the sum of exponentials of first 3 Fibonacci numbers",
)

print(response.text)

Let $F_n$ be the $n$-th Fibonacci number, with $F_1 = 1, F_2 = 1, F_3 = 2$.
We want to calculate the sum of exponentials of the first 3 Fibonacci numbers, which is
$$ e^{F_1} + e^{F_2} + e^{F_3} = e^1 + e^1 + e^2 = e + e + e^2 = 2e + e^2 $$
We can approximate $e \approx 2.71828$, so
$$ 2e + e^2 \approx 2(2.71828) + (2.71828)^2 \approx 5.43656 + 7.38905 \approx 12.82561 $$
Alternatively,
$$ 2e + e^2 = e(2+e) \approx 2.71828(2 + 2.71828) = 2.71828(4.71828) \approx 12.82561 $$

The sum of the exponentials of the first 3 Fibonacci numbers is $e^1 + e^1 + e^2 = e + e + e^2 = 2e + e^2$.

Final Answer: The final answer is $\boxed{2e+e^2}$


In [ ]:
# Get model's response
system_prompt = '''You are a math agent. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input

where python_function_name is one of the following:
1. strings_to_chars_to_int(string) - Takes a word as input and returns ASCII INT values of characters in the word as a list.
2. int_list_to_exponential_sum(list) - Takes a list of integers and returns the sum of exponentials of those integers.
3. fibonacci_numbers(int) - Takes an integer, like 6, and returns the first 6 integers in a Fibonacci series as a list.

DO NOT include multiple responses. Give ONE response at a time. DO NOT compute anything yourself.'''
"""You are a math agent. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input
2. FINAL_ANSWER: [number]

where python_function_name is one of the followin:
1. strings_to_chars_to_int(string) It takes a word as input, and returns the ASCII INT values of characters in the word as a list
2. int_list_to_exponential_sum(list) It takes a list of integers and returns the sum of exponentials of those integers
3. fibonacci_numbers(int) It takes an integer, like 6, and returns first 6 integers in a fibonacci series as a list.
DO NOT include multiple responses. Give ONE response at a time."""

current_query= """Calculate the sum of exponentials of word "TSAI"""

prompt = f"{system_prompt}\n\nQuery: {current_query}"
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

print(response.text)

In [ ]:
import math

def strings_to_chars_to_int(string):
    return [ord(char) for char in string]

def int_list_to_exponential_sum(int_list):
    int_list = eval(int_list)
    return sum(math.exp(i) for i in int_list)

def fibonacci_numbers(n):
    if n <= 0:
        return []
    fib_sequence = [0, 1]
    for _ in range(2, n):
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence[:n]

In [ ]:
response_text = response.text.strip()
response_text

In [ ]:
_, function_info = response_text.split(":", 1)
_, function_info

In [ ]:
func_name, params = [x.strip() for x in function_info.split("|", 1)]

func_name, params

In [ ]:
def function_caller(func_name, params):
    """Simple function caller that maps function names to actual functions"""
    function_map = {
        "strings_to_chars_to_int": strings_to_chars_to_int,
        "int_list_to_exponential_sum": int_list_to_exponential_sum,
        "fibonacci_numbers": fibonacci_numbers
    }
    
    if func_name in function_map:
        return function_map[func_name](params)
    else:
        return f"Function {func_name} not found"

In [ ]:
iteration_result = function_caller(func_name, params)

In [ ]:
# Get model's response
system_prompt = '''You are a math agent. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input

where python_function_name is one of the following:
1. strings_to_chars_to_int(string) - Takes a word as input and returns ASCII INT values of characters in the word as a list.
2. int_list_to_exponential_sum(list) - Takes a list of integers and returns the sum of exponentials of those integers.
3. fibonacci_numbers(int) - Takes an integer, like 6, and returns the first 6 integers in a Fibonacci series as a list.

DO NOT include multiple responses. Give ONE response at a time. DO NOT compute anything yourself.'''
"""You are a math agent solving problems in iterations. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input
2. FINAL_ANSWER: [number]

where python_function_name is one of the followin:
1. strings_to_chars_to_int(string) It takes a word as input, and returns the ASCII INT values of characters in the word as a list
2. int_list_to_exponential_sum(list) It takes a list of integers and returns the sum of exponentials of those integers
3. fibonacci_numbers(int) It takes an integer, like 6, and returns first 6 integers in a fibonacci series as a list.
DO NOT include multiple responses. Give ONE response at a time."""

current_query= """Calculate the sum of exponentials of word "TSAI"""
iteration_1 = f"In the first iteration you called {func_name} with {params} parameters, and the function returned {iteration_result}. What should I do next?"

prompt = f"{system_prompt}\n\nQuery: {current_query}\n\n{iteration_1}"
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

print(response.text)

In [ ]:
response_text = response.text.strip()
_, function_info = response_text.split(":", 1)
func_name, params = [x.strip() for x in function_info.split("|", 1)]
iteration_result = function_caller(func_name, params)
iteration_result

In [ ]:
# Get model's response
system_prompt = '''You are a math agent. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input

where python_function_name is one of the following:
1. strings_to_chars_to_int(string) - Takes a word as input and returns ASCII INT values of characters in the word as a list.
2. int_list_to_exponential_sum(list) - Takes a list of integers and returns the sum of exponentials of those integers.
3. fibonacci_numbers(int) - Takes an integer, like 6, and returns the first 6 integers in a Fibonacci series as a list.

DO NOT include multiple responses. Give ONE response at a time. DO NOT compute anything yourself.'''
"""You are a math agent solving problems in iterations. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input
2. FINAL_ANSWER: [number]

where python_function_name is one of the followin:
1. strings_to_chars_to_int(string) It takes a word as input, and returns the ASCII INT values of characters in the word as a list
2. int_list_to_exponential_sum(list) It takes a list of integers and returns the sum of exponentials of those integers
3. fibonacci_numbers(int) It takes an integer, like 6, and returns first 6 integers in a fibonacci series as a list.
DO NOT include multiple responses. Give ONE response at a time."""

current_query= """Calculate the sum of exponentials of word "TSAI"""
iteration_1 = f"In the first iteration you called strings_to_chars_to_int with TSAI parameters, and the function returned {iteration_result}. What should I do next?"
iteration_2 = f"In the first iteration you called {func_name} with {params} parameters, and the function returned {iteration_result}. What should I do next?"
prompt = f"{system_prompt}\n\nQuery: {current_query}\n\n{iteration_1}\n\n{iteration_2}"
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

print(response.text)

In [ ]:
max_iterations = 3
last_response = None
iteration = 0
iteration_response = []

system_prompt = '''You are a math agent. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input

where python_function_name is one of the following:
1. strings_to_chars_to_int(string) - Takes a word as input and returns ASCII INT values of characters in the word as a list.
2. int_list_to_exponential_sum(list) - Takes a list of integers and returns the sum of exponentials of those integers.
3. fibonacci_numbers(int) - Takes an integer, like 6, and returns the first 6 integers in a Fibonacci series as a list.

DO NOT include multiple responses. Give ONE response at a time. DO NOT compute anything yourself.'''
"""You are a math agent solving problems in iterations. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input
2. FINAL_ANSWER: [number]

where python_function_name is one of the followin:
1. strings_to_chars_to_int(string) It takes a word as input, and returns the ASCII INT values of characters in the word as a list
2. int_list_to_exponential_sum(list) It takes a list of integers and returns the sum of exponentials of those integers
3. fibonacci_numbers(int) It takes an integer, like 6, and returns first 6 integers in a fibonacci series as a list.
DO NOT include multiple responses. Give ONE response at a time."""

query= """Calculate the sum of exponentials of word "TSAI"""

while iteration < max_iterations:
    print(f"\n--- Iteration {iteration + 1} ---")
    if last_response == None:
        current_query = query
    else:
        current_query = current_query + "\n\n" + " ".join(iteration_response)
        current_query = current_query + "  What should I do next?"

    # Get model's response
    prompt = f"{system_prompt}\n\nQuery: {current_query}"
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    response_text = response.text.strip()
    print(f"LLM Response: {response_text}")

    
    if response_text.startswith("FUNCTION_CALL:"):
        response_text = response.text.strip()
        _, function_info = response_text.split(":", 1)
        func_name, params = [x.strip() for x in function_info.split("|", 1)]
        iteration_result = function_caller(func_name, params)

    # Check if it's the final answer
    elif response_text.startswith("FINAL_ANSWER:"):
        print("\n=== Agent Execution Complete ===")
        break
        

    print(f"  Result: {iteration_result}")
    last_response = iteration_result
    iteration_response.append(f"In the {iteration + 1} iteration you called {func_name} with {params} parameters, and the function returned {iteration_result}.")

    iteration += 1
    
    


In [ ]:
max_iterations = 3
last_response = None
iteration = 0
iteration_response = []

system_prompt =  """You are a math agent solving problems in iterations. Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: python_function_name|input
2. FINAL_ANSWER: [number]

where python_function_name is one of the following:
1. strings_to_chars_to_int(string) It takes a word as input, and returns the ASCII INT values of characters in the word as a list
2. int_list_to_exponential_sum(list) It takes a list of integers and returns the sum of exponentials of those integers
3. fibonacci_numbers(int) It takes an integer, like 6, and returns first 6 integers in a fibonacci series as a list.
DO NOT include multiple responses. Give ONE response at a time."""

query= """Calculate the sum of exponentials of word "TSAI"""

while iteration < max_iterations:
    print(f"\n--- Iteration {iteration + 1} ---")
    if last_response == None:
        current_query = query
    else:
        current_query = current_query + "\n\n" + " ".join(iteration_response)
        current_query = current_query + "  What should I do next?"

    # Get model's response
    prompt = f"{system_prompt}\n\nQuery: {current_query}"
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    response_text = response.text.strip()
    print(f"LLM Response: {response_text}")

    
    if response_text.startswith("FUNCTION_CALL:"):
        response_text = response.text.strip()
        _, function_info = response_text.split(":", 1)
        func_name, params = [x.strip() for x in function_info.split("|", 1)]
        iteration_result = function_caller(func_name, params)

    # Check if it's the final answer
    elif response_text.startswith("FINAL_ANSWER:"):
        print("\n=== Agent Execution Complete ===")
        break
        

    print(f"  Result: {iteration_result}")
    last_response = iteration_result
    iteration_response.append(f"In the {iteration + 1} iteration you called {func_name} with {params} parameters, and the function returned {iteration_result}.")

    iteration += 1
    
    


In [5]:
import os
# import google.generativeai as genai
import requests
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Local API for fetching OTT recommendations
LOCAL_API_URL = "http://127.0.0.1:5000/recommend"
from google import genai

client = genai.Client(api_key=api_key)

# response = client.models.generate_content(
#     model="gemini-2.0-flash",
#     contents="Calculate the sum of exponentials of first 3 Fibonacci numbers",
# )


In [6]:
def extract_ott_platform(query):
    """Extracts OTT platform from query and returns standardized name."""
    ott_platforms = {
        "netflix": "Netflix",
        "amazon prime": "Amazon Prime Video",
        "hulu": "Hulu",
        "disney+": "Disney+",
        "hbo max": "HBO Max"
    }
    
    query_lower = query.lower()
    for key, platform in ott_platforms.items():
        if key in query_lower:
            return platform

    return "Netflix"  # Default to Netflix


In [11]:
def get_series_recommendations(platform):
    """Calls local API to fetch top OTT series recommendations."""
    api_url = f"{LOCAL_API_URL}?platform={platform.replace(' ', '%20')}"
    response = requests.get(api_url)

    if response.status_code == 200:
        return response.json().get("recommendations", [])
    
    return []


In [28]:
def format_recommendations(recommendations):
    if not isinstance(recommendations, list):  # Ensure response is a list
        return "Error: Invalid recommendations format received."

    if not recommendations:
        return "No recommendations found."

    formatted_result = "\n\n".join(
        [
            f"🎬 *{rec.get('title', 'Unknown')}* ({rec.get('rating', 'N/A')}⭐)\n"
            f"📖 {rec.get('description', 'No description available')}\n"
            f"🖼️ [Poster]({rec.get('poster', '#')})"
            for rec in recommendations if isinstance(rec, dict)
        ]
    )

    return formatted_result


In [9]:
FUNCTION_MAPPING = {
    "extract_ott_platform": extract_ott_platform,
    "get_series_recommendations": get_series_recommendations,
    "format_recommendations": format_recommendations
}


In [27]:
import json  # Required for parsing JSON parameters

def function_caller(func_name, params):
    FUNCTION_MAPPING = {
        "extract_ott_platform": extract_ott_platform,
        "get_series_recommendations": get_series_recommendations,
        "format_recommendations": format_recommendations
    }

    if func_name in FUNCTION_MAPPING:
        # Convert params if needed
        try:
            parsed_params = json.loads(params)  # Convert string to list/dict if possible
        except json.JSONDecodeError:
            parsed_params = params  # Keep as string if not JSON formatted

        return FUNCTION_MAPPING[func_name](parsed_params)
    else:
        return f"Function {func_name} not found"


In [29]:
query = "I need something good to watch on Netflix"
platform = extract_ott_platform(query)
print("Extracted Platform:", platform)

recommendations = get_series_recommendations(platform)
print("Raw Recommendations:", recommendations)

formatted_result = format_recommendations(recommendations)
print("Formatted Recommendations:\n", formatted_result)


Extracted Platform: Netflix
Raw Recommendations: [{'description': 'When a 13-year-old is accused of the murder of a classmate, his family, therapist and the detective in charge are all left asking: what really happened?', 'poster': 'https://image.tmdb.org/t/p/w200/tDHWWReefmOOjBCJZUck8cNwssk.jpg', 'rating': 7.8, 'title': 'Adolescence'}, {'description': 'Robin, Starfire, Raven, Beast Boy and Cyborg return in all-new, comedic adventures. They may be super heroes who save the world every day ... but somebody still has to do the laundry!', 'poster': 'https://image.tmdb.org/t/p/w200/udaLIJ6Na7GOHjvTlyP9JFPTccv.jpg', 'rating': 6.5, 'title': 'Teen Titans Go!'}, {'description': "Rick is a mentally-unbalanced but scientifically gifted old man who has recently reconnected with his family. He spends most of his time involving his young grandson Morty in dangerous, outlandish adventures throughout space and alternate universes. Compounded with Morty's already unstable family life, these events cau

In [30]:
max_iterations = 3
last_response = None
iteration = 0
history = []

system_prompt = """You are an OTT recommendation agent solving user queries in iterations. 
Respond with EXACTLY ONE of these formats:
1. FUNCTION_CALL: extract_ott_platform|query
2. FUNCTION_CALL: get_series_recommendations|platform
3. FUNCTION_CALL: format_recommendations|recommendations
4. FINAL_ANSWER: formatted_output

DO NOT include multiple responses. Give ONE response at a time."""
query = "I need something good to watch on Netflix"


while iteration < max_iterations:
    print(f"\n--- Iteration {iteration + 1} ---")
    if last_response == None:
        current_query = query
    else:
        current_query = current_query + "\n\n" + " ".join(history)
        current_query = current_query + "  What should I do next?"

    # Get model's response
    prompt = f"{system_prompt}\n\nQuery: {current_query}"
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    
    response_text = response.text.strip()
    print(f"LLM Response: {response_text}")

    
    if response_text.startswith("FUNCTION_CALL:"):
        response_text = response.text.strip()
        _, function_info = response_text.split(":", 1)
        func_name, params = [x.strip() for x in function_info.split("|", 1)]
        iteration_result = function_caller(func_name, params)

    # Check if it's the final answer
    elif response_text.startswith("FINAL_ANSWER:"):
        print("\n=== Agent Execution Complete ===")
        break
        

    print(f"  Result: {iteration_result}")
    last_response = iteration_result
    history.append(f"In the {iteration + 1} iteration you called {func_name} with {params} parameters, and the function returned {iteration_result}.")

    iteration += 1
    
    



--- Iteration 1 ---
LLM Response: FUNCTION_CALL: extract_ott_platform|query
  Result: Netflix

--- Iteration 2 ---
LLM Response: FUNCTION_CALL: get_series_recommendations|Netflix
  Result: [{'description': 'When a 13-year-old is accused of the murder of a classmate, his family, therapist and the detective in charge are all left asking: what really happened?', 'poster': 'https://image.tmdb.org/t/p/w200/tDHWWReefmOOjBCJZUck8cNwssk.jpg', 'rating': 7.8, 'title': 'Adolescence'}, {'description': 'Robin, Starfire, Raven, Beast Boy and Cyborg return in all-new, comedic adventures. They may be super heroes who save the world every day ... but somebody still has to do the laundry!', 'poster': 'https://image.tmdb.org/t/p/w200/udaLIJ6Na7GOHjvTlyP9JFPTccv.jpg', 'rating': 6.5, 'title': 'Teen Titans Go!'}, {'description': "Rick is a mentally-unbalanced but scientifically gifted old man who has recently reconnected with his family. He spends most of his time involving his young grandson Morty in dang

In [ ]:
--- Iteration 1 ---
LLM Response: FUNCTION_CALL: extract_ott_platform|query
  Result: Netflix

--- Iteration 2 ---
LLM Response: FUNCTION_CALL: get_series_recommendations|Netflix
  Result: [{'description': 'When a 13-year-old is accused of the murder of a classmate, his family, therapist and the detective in charge are all left asking: what really happened?', 'poster': 'https://image.tmdb.org/t/p/w200/tDHWWReefmOOjBCJZUck8cNwssk.jpg', 'rating': 7.8, 'title': 'Adolescence'}, {'description': 'Robin, Starfire, Raven, Beast Boy and Cyborg return in all-new, comedic adventures. They may be super heroes who save the world every day ... but somebody still has to do the laundry!', 'poster': 'https://image.tmdb.org/t/p/w200/udaLIJ6Na7GOHjvTlyP9JFPTccv.jpg', 'rating': 6.5, 'title': 'Teen Titans Go!'}, {'description': "Rick is a mentally-unbalanced but scientifically gifted old man who has recently reconnected with his family. He spends most of his time involving his young grandson Morty in dangerous, outlandish adventures throughout space and alternate universes. Compounded with Morty's already unstable family life, these events cause Morty much distress at home and school.", 'poster': 'https://image.tmdb.org/t/p/w200/gdIrmf2DdY5mgN6ycVP0XlzKzbE.jpg', 'rating': 8.687, 'title': 'Rick and Morty'}, {'description': 'A reality series that follows some of the most affluent women in the country as they enjoy the lavish lifestyle that only Beverly Hills can provide.', 'poster': 'https://image.tmdb.org/t/p/w200/gHZmEmtQzobW9PVdpGrYP7SU9RH.jpg', 'rating': 6.3, 'title': 'The Real Housewives of Beverly Hills'}, {'description': 'Raymond "Red" Reddington, one of the FBI\'s most wanted fugitives, surrenders in person at FBI Headquarters in Washington, D.C. He claims that he and the FBI have the same interests: bringing down dangerous criminals and terrorists. In the last two decades, he\'s made a list of criminals and terrorists that matter the most but the FBI cannot find because it does not know they exist. Reddington calls this "The Blacklist". Reddington will co-operate, but insists that he will speak only to Elizabeth Keen, a rookie FBI profiler.', 'poster': 'https://image.tmdb.org/t/p/w200/4HTfd1PhgFUenJxVuBDNdLmdr0c.jpg', 'rating': 7.632, 'title': 'The Blacklist'}]

--- Iteration 3 ---
LLM Response: FUNCTION_CALL: format_recommendations|recommendations
Formatted Recommendations:
 🎬 *Adolescence* (7.8⭐)
📖 When a 13-year-old is accused of the murder of a classmate, his family, therapist and the detective in charge are all left asking: what really happened?
🖼️ [Poster](https://image.tmdb.org/t/p/w200/tDHWWReefmOOjBCJZUck8cNwssk.jpg)

🎬 *Teen Titans Go!* (6.5⭐)
📖 Robin, Starfire, Raven, Beast Boy and Cyborg return in all-new, comedic adventures. They may be super heroes who save the world every day ... but somebody still has to do the laundry!
🖼️ [Poster](https://image.tmdb.org/t/p/w200/udaLIJ6Na7GOHjvTlyP9JFPTccv.jpg)

🎬 *Rick and Morty* (8.687⭐)
📖 Rick is a mentally-unbalanced but scientifically gifted old man who has recently reconnected with his family. He spends most of his time involving his young grandson Morty in dangerous, outlandish adventures throughout space and alternate universes. Compounded with Morty's already unstable family life, these events cause Morty much distress at home and school.
🖼️ [Poster](https://image.tmdb.org/t/p/w200/gdIrmf2DdY5mgN6ycVP0XlzKzbE.jpg)

🎬 *The Real Housewives of Beverly Hills* (6.3⭐)
📖 A reality series that follows some of the most affluent women in the country as they enjoy the lavish lifestyle that only Beverly Hills can provide.
🖼️ [Poster](https://image.tmdb.org/t/p/w200/gHZmEmtQzobW9PVdpGrYP7SU9RH.jpg)

🎬 *The Blacklist* (7.632⭐)
📖 Raymond "Red" Reddington, one of the FBI's most wanted fugitives, surrenders in person at FBI Headquarters in Washington, D.C. He claims that he and the FBI have the same interests: bringing down dangerous criminals and terrorists. In the last two decades, he's made a list of criminals and terrorists that matter the most but the FBI cannot find because it does not know they exist. Reddington calls this "The Blacklist". Reddington will co-operate, but insists that he will speak only to Elizabeth Keen, a rookie FBI profiler.
🖼️ [Poster](https://image.tmdb.org/t/p/w200/4HTfd1PhgFUenJxVuBDNdLmdr0c.jpg)